In [ ]:
import numpy as np
from numpy.linalg import inv, norm
from scipy.sparse.linalg import cg
from sklearn import metrics
from sklearn.metrics import matthews_corrcoef
from combss_group_functions import *
from cd import cd_swaps

Below cells set parameters for data and generate data

In [ ]:
n = 400
beta_type = 2
#Number of active groups
active = 15
#Number of groups
m = 150
# Group size
vrank = 4
p = m*vrank
#Signal-to-noise ratio
snr = 3
#Within-group correlation
ro = 0.9
#Between-group correlation
psi = 0.5
#Generate true coefficients 
beta0 =  gen_beta0gr(p, active*vrank,m,vrank, beta_type)
true_coefs = beta0

In [ ]:
%%time
#Generating Data
blc = np.ones((vrank,vrank))*ro
covX = block_diag((blc,)*m)
covX.setdiag(1)
covX = covX.toarray()
covX[covX==0] = psi
noise_var = (beta0.T@covX@beta0/snr)
true_groups = true_coefs[::vrank]
group_indices = [[vrank*i + j for j in range(vrank)] for i in range(int(p/vrank))]
L = cholesky(covX, lower=True)
X = L@np.random.normal(0,1,(p,n))
X = X.T
y = X@beta0 + np.random.normal(0, 1, size=n)*np.sqrt(noise_var)

The below cell is an example of using L0Group, see, https://github.com/hazimehh/L0Group

In [ ]:
%%time
"""This cell demonstrates how to run the CD + Local Search algorithm."""
Params = {
    'NumIters': 100000, # maximum number of CD full cycles.
    'Tolerance': 1e-7 , # convergence tolerance.
    'Logging': False, # prints output while running
    'FeaturesOrder': 'Cyclic', # order in which CD updates the variables.
}

ld_cd = 1800
ldr_cd = 1000

cd_swaps_object = cd_swaps(X, y, group_indices, Params)
# The fit function takes the same arguments as that of CD, in addition to "max_swaps" 
# which sets the maximum number of group swaps the local search algorithms can perform.
#lamba_0 is the l0 lambda penalty, lambda_2 is the ridge lambda penalty
tht_cd, obj = cd_swaps_object.fit(lambda_0 = ld_cd, lambda_2 = ldr_cd, max_swaps=100, warm_start=np.zeros(X.shape[1]))
print("Objective: ", obj)
print("Solution: ", tht_cd.reshape(m,vrank))
print("Number of Selected Groups: ", sum(abs(tht_cd[::vrank])>0))

The below cell is example of using group COMBSS

In [ ]:
%%time
#COMBSS Implementation

#Algorithm parameters
ap = 1
alpha = 0.1
max_iter = 10**3 
tau = 0.1

#l1 penalty
ld = 2200

#ridge penalty
ldr = 1000

tm = 0.5*np.ones(m)
combss_t, tht = combss_adam(y, X, n, p, m, vrank, tm, ld/n, ldr, ap, alpha, max_iter)
#tht below is the same as \tilde{theta_t}
tht = tht.reshape(m,vrank)
print("Solution: " , tht.reshape(m,vrank))
print("Number of Selected Groups: ", sum(combss_t>tau))

Now we will run simulations comparing COMBSS vs COMBSS (with augmented data) vs L0Group(MIO) vs Greedy

In [ ]:
# The number of simulations
nsim = 100
#Grid size for the L0/L1 penalty
gdsize = 100
#Grid size for the L2 penalty
gdsize_l2 = 10

In [ ]:
#Initialize grids for the 3 different methods
grid_combss_l1 = np.flip(np.round(np.linspace(10, 10000, num=gdsize),2))
grid_combss_l2 = np.flip(np.round(np.linspace(0, 4000, num=gdsize_l2),2))

grid_combssaug_l1 = np.flip(np.round(np.linspace(10, 20000, num=gdsize),2))
grid_combssaug_l2 = np.flip(np.round(np.linspace(0, 500, num=gdsize_l2),2))

grid_mio_l1 = np.flip(np.round(np.linspace(10, 10000, num=gdsize),2))
grid_mio_l2 = np.flip(np.round(np.linspace(0, 1000, num=gdsize_l2),2))

grid_greedy_l2 = np.flip(np.round(np.linspace(0, 2500, num=gdsize_l2),2))

In [ ]:
#Initialize arrays for storing metrics 

#MIO
tp_mio = np.zeros(shape=(nsim,gdsize,gdsize_l2))
b0_mio = np.zeros(shape=(nsim,gdsize,gdsize_l2))
error_mio = np.zeros(shape=(nsim,gdsize,gdsize_l2))
errortest_mio = np.zeros(shape=(nsim,gdsize,gdsize_l2))
mcc_mio = np.zeros(shape=(nsim,gdsize,gdsize_l2))
fp_mio = np.zeros(shape=(nsim,gdsize,gdsize_l2))
bias_inf_mio = np.zeros(shape=(nsim,gdsize,gdsize_l2))
bias_l2_mio = np.zeros(shape=(nsim,gdsize,gdsize_l2))
errortestsamp_mio = np.zeros(shape=(nsim,gdsize,gdsize_l2))


#COMBSS
tp_combss = np.zeros(shape=(nsim,gdsize,gdsize_l2))
b0_combss = np.zeros(shape=(nsim,gdsize,gdsize_l2))
error_combss = np.zeros(shape=(nsim,gdsize,gdsize_l2))
errortest_combss = np.zeros(shape=(nsim,gdsize,gdsize_l2))
errortestend_combss = np.zeros(shape=(nsim,gdsize,gdsize_l2))
mcc_combss = np.zeros(shape=(nsim,gdsize,gdsize_l2))
fp_combss = np.zeros(shape=(nsim,gdsize,gdsize_l2))
bias_inf_combss = np.zeros(shape=(nsim,gdsize,gdsize_l2))
biasend_inf_combss = np.zeros(shape=(nsim,gdsize,gdsize_l2))
bias_l2_combss = np.zeros(shape=(nsim,gdsize,gdsize_l2))
biasend_l2_combss = np.zeros(shape=(nsim,gdsize,gdsize_l2))
errortestsamp_combss = np.zeros(shape=(nsim,gdsize,gdsize_l2))
errortestendsamp_combss = np.zeros(shape=(nsim,gdsize,gdsize_l2))

#COMBSS AUGMENTED
tp_combssaug = np.zeros(shape=(nsim,gdsize,gdsize_l2))
b0_combssaug = np.zeros(shape=(nsim,gdsize,gdsize_l2))
error_combssaug  = np.zeros(shape=(nsim,gdsize,gdsize_l2))
errortest_combssaug  = np.zeros(shape=(nsim,gdsize,gdsize_l2))
errortestend_combssaug  = np.zeros(shape=(nsim,gdsize,gdsize_l2))
mcc_combssaug = np.zeros(shape=(nsim,gdsize,gdsize_l2))
fp_combssaug = np.zeros(shape=(nsim,gdsize,gdsize_l2))
bias_inf_combssaug = np.zeros(shape=(nsim,gdsize,gdsize_l2))
biasend_inf_combssaug = np.zeros(shape=(nsim,gdsize,gdsize_l2))
bias_l2_combssaug = np.zeros(shape=(nsim,gdsize,gdsize_l2))
biasend_l2_combssaug = np.zeros(shape=(nsim,gdsize,gdsize_l2))
errortestsamp_combssaug = np.zeros(shape=(nsim,gdsize,gdsize_l2))
errortestendsamp_combssaug = np.zeros(shape=(nsim,gdsize,gdsize_l2))


#Greedy
tp_greedy = np.zeros(shape=(nsim,1,gdsize_l2))
#no_need_for_b0_in_greedy
error_greedy = np.zeros(shape=(nsim,1,gdsize_l2))
errortest_greedy = np.zeros(shape=(nsim,1,gdsize_l2))
mcc_greedy = np.zeros(shape=(nsim,1,gdsize_l2))
fp_greedy = np.zeros(shape=(nsim,1,gdsize_l2))
bias_greedy_inf = np.zeros(shape=(nsim,1,gdsize_l2))
bias_greedy_l2 = np.zeros(shape=(nsim,1,gdsize_l2))
errortestsamp_greedy = np.zeros(shape=(nsim,1,gdsize_l2))

Below cell will run experiments

In [ ]:
%%time
b0_max = 75
ntest = n*10
for i in range(nsim):
    #Generate data
    X = L@np.random.normal(0,1,(p,n))
    X = X.T
    Xtest = L@np.random.normal(0,1,(p,ntest))
    Xtest = Xtest.T
    y = X@beta0 + np.random.normal(0, 1, size=n)*np.sqrt(noise_var)
    true_groups = beta0[::vrank]
    group_indices = [[vrank*i + j for j in range(vrank)] for i in range(int(p/vrank))]
              
    #COMBSS
    for k in range(gdsize_l2):
        for j in range(gdsize):
            #print(j)
            tm = 0.5*np.ones(m)
            ld = grid_combss_l1[j]
            ldr = grid_combss_l2[k]
            combss_t,tht = combss_adam(y,X, n, p, m, vrank,tm,ld/n,ldr,ap,alpha,max_iter)
            combss_s = combss_t>0.1
            b0_combss[i,j,k] = sum(combss_s) 
            error_combss[i,j,k] = get_error(y, X,vrank,  combss_s,ldr)
            tp_combss[i,j,k] = sum(np.isin(np.where( combss_s)[0],np.where(true_groups>0)[0]))
            mcc_combss[i,j,k] = matthews_corrcoef(true_groups, combss_s)
            fp_combss[i,j,k] = b0_combss[i,j,k]-tp_combss[i,j,k]
            res = beta0-tht
            resend = beta0-get_theta(y, X,vrank,  combss_s,ldr)
            errortestend_combss[i,j,k] = np.linalg.norm(X@resend)
            errortest_combss[i,j,k] = np.linalg.norm(X@res)
            errortestendsamp_combss[i,j,k] = np.linalg.norm(Xtest@resend)
            errortestsamp_combss[i,j,k] = np.linalg.norm(Xtest@res)            
            bias_inf_combss[i,j,k] = np.linalg.norm(res,np.inf)
            biasend_inf_combss[i,j,k] = np.linalg.norm(resend,np.inf)
            bias_l2_combss[i,j,k] = np.linalg.norm(res)
            biasend_l2_combss[i,j,k] =np.linalg.norm(resend)

            if b0_combss[i,j,k]>=b0_max:
                break
                
    #AUGMENTED COMBSS
    for k in range(gdsize_l2):
        #print(k)
        for j in range(gdsize):          
            tm = 0.5*np.ones(m)
            ld = grid_combssaug_l1[j]
            ldr_aug = grid_combssaug_l2[k]
            Xnew = np.append(X,np.sqrt(ldr_aug)*np.eye(p),0)
            ynew = np.append(y, np.zeros(p))
            combssaug_t,tht = combss_adam(ynew,Xnew, n, p, m, vrank,tm,ld/n,0,ap,alpha,max_iter)  
            combssaug_s = combssaug_t>0.1
            b0_combssaug[i,j,k] = sum(combssaug_s) 
            error_combssaug[i,j,k] = get_error(y, X, vrank, combssaug_s,ldr_aug)
            tp_combssaug[i,j,k] = sum(np.isin(np.where(combssaug_s)[0],np.where(true_groups>0)[0]))
            mcc_combssaug[i,j,k] = matthews_corrcoef(true_groups,combssaug_s)
            fp_combssaug[i,j,k] = b0_combssaug[i,j,k]-tp_combssaug[i,j,k]
            res = beta0-tht
            resend = beta0-get_theta(y, X, vrank, combssaug_s,ldr_aug)
            errortestend_combssaug[i,j,k] = np.linalg.norm(X@resend)
            errortest_combssaug[i,j,k] = np.linalg.norm(X@res)
            errortestendsamp_combssaug[i,j,k] = np.linalg.norm(Xtest@resend)
            errortestsamp_combssaug[i,j,k] = np.linalg.norm(Xtest@res)
            bias_inf_combssaug[i,j,k] = np.linalg.norm(res,np.inf)
            biasend_inf_combssaug[i,j,k] = np.linalg.norm(resend,np.inf)
            bias_l2_combssaug[i,j,k] = np.linalg.norm(res)
            biasend_l2_combssaug[i,j,k] =np.linalg.norm(resend)
            
            if b0_combssaug[i,j,k]>=b0_max:
                break
    #GREEDY
    for k in range(gdsize_l2):        
        gre_ldr = grid_greedy_l2[k]
        greedy_sol = greedy_method(active, X, vrank,y, m, gre_ldr)
        error_greedy[i,0,k] = get_error(y, X, vrank, greedy_sol, gre_ldr)
        tp_greedy[i,0,k] =   sum(np.isin(np.where(greedy_sol>0.1)[0],np.where(true_groups>0)[0]))
        mcc_greedy[i,0,k] = matthews_corrcoef(true_groups,greedy_sol)
        fp_greedy[i,0,k] = active- tp_greedy[i,0,k]
        res = beta0-get_theta(y, X, vrank, greedy_sol, gre_ldr)
        errortest_greedy[i,0,k] = np.linalg.norm(X@res)
        errortestsamp_greedy[i,0,k] = np.linalg.norm(Xtest@res)
        bias_greedy_l2[i,0,k] = np.linalg.norm(res)
        bias_greedy_inf[i,0,k] = np.linalg.norm(res,np.inf)
        
    
                                    
    #MIO
    for k in range(gdsize_l2):
        for j in range(gdsize):
            #print(j)
            ld = grid_mio_l1[j]
            ldr = grid_mio_l2[k]
            cd_swaps_object = cd_swaps(X, y, group_indices, Params)
            tht_cd, obj = cd_swaps_object.fit(lambda_0= ld, lambda_2=ldr, max_swaps=100, warm_start=np.zeros(X.shape[1]))
            cd_s = abs(tht_cd[::vrank])>0
            b0_mio[i,j,k] = sum(cd_s)
            error_mio[i,j,k] = get_error(y, X, vrank, cd_s,ldr)
            tp_mio[i,j,k] =  sum(np.isin(np.where(cd_s)[0],np.where(true_groups>0)[0]))
            mcc_mio[i,j,k] = matthews_corrcoef(true_groups,cd_s)
            fp_mio[i,j,k] = b0_mio[i,j,k] - tp_mio[i,j,k]
            res = beta0-tht_cd
            errortest_mio[i,j,k] = np.linalg.norm(X@res)
            errortestsamp_mio[i,j,k] = np.linalg.norm(Xtest@res)
            bias_inf_mio[i,j,k] =  np.linalg.norm(res,np.inf)
            bias_l2_mio[i,j,k] = np.linalg.norm(res)
            
            if b0_mio[i,j,k]>=b0_max:
                break
        
        
    print("Completed sim : ", i)

    
    

In [ ]:
#Print results for COMBSS using the endpoint

b0s = []
mccs = []
tps = []
fps = []
bias_infs = []
biasl2s = []
tests = []
ridges = []
testsamps = []

for simi in range(nsim):
    temp = errortestendsamp_combss[simi,:,:]
    temp_indx = np.where(temp == np.min(temp[temp>0]))
    b0s.append(b0_combss[simi,temp_indx[0],temp_indx[1]][0])
    mccs.append(mcc_combss[simi,temp_indx[0],temp_indx[1]][0])
    tps.append(tp_combss[simi,temp_indx[0],temp_indx[1]][0])
    fps.append(fp_combss[simi,temp_indx[0],temp_indx[1]][0])
    bias_infs.append(biasend_inf_combss[simi,temp_indx[0],temp_indx[1]][0])
    biasl2s.append(biasend_l2_combss[simi,temp_indx[0],temp_indx[1]][0])
    tests.append(np.min(temp[temp>0]))
    ridges.append(grid_combss_l2[temp_indx[1]][0])
    testsamps.append(errortestendsamp_combss[simi,temp_indx[0],temp_indx[1]][0])

labels = ["\|B\|_0", "MCC", "TPs", "FPs", "Bias (L2)", "Test Error", "Optimal Ridge Penalty" ]
j = 0
for i in [b0s,mccs,tps,fps ,biasl2s,tests,ridges]:
    prnt_elem = i
    print(labels[j]+": "+str(np.round(np.mean(prnt_elem),2))+" "+"("+str(np.round(np.sqrt(np.var(prnt_elem)/nsim),2))+")")
    j = j+1

In [ ]:
#Print results for COMBSS using \tilde{theta_t} where t is the convergent point in GD.

b0s = []
mccs = []
tps = []
fps = []
bias_infs = []
biasl2s = []
tests = []
ridges = []
testsamps = []

for simi in range(nsim):
    temp = errortestsamp_combss[simi,:,:]
    temp_indx = np.where(temp == np.min(temp[temp>0]))
    b0s.append(b0_combss[simi,temp_indx[0],temp_indx[1]][0])
    mccs.append(mcc_combss[simi,temp_indx[0],temp_indx[1]][0])
    tps.append(tp_combss[simi,temp_indx[0],temp_indx[1]][0])
    fps.append(fp_combss[simi,temp_indx[0],temp_indx[1]][0])
    bias_infs.append(bias_inf_combss[simi,temp_indx[0],temp_indx[1]][0])
    biasl2s.append(bias_l2_combss[simi,temp_indx[0],temp_indx[1]][0])
    tests.append(np.min(temp[temp>0]))
    ridges.append(grid_combss_l2[temp_indx[1]][0])
    testsamps.append(errortestsamp_combss[simi,temp_indx[0],temp_indx[1]][0])

labels = ["\|B\|_0", "MCC", "TPs", "FPs", "Bias (L2)", "Test Error", "Optimal Ridge Penalty" ]
j = 0
for i in [b0s,mccs,tps,fps ,biasl2s,tests,ridges]:
    prnt_elem = i
    print(labels[j]+": "+str(np.round(np.mean(prnt_elem),2))+" "+"("+str(np.round(np.sqrt(np.var(prnt_elem)/nsim),2))+")")
    j = j+1

In [ ]:
#Print results for COMBSS augmented using the endpoint

b0s = []
mccs = []
tps = []
fps = []
bias_infs = []
biasl2s = []
tests = []
ridges = []
testsamps = []

for simi in range(nsim):
    temp = errortestendsamp_combssaug[simi,:,:]
    temp_indx = np.where(temp == np.min(temp[temp>0]))
    b0s.append(b0_combssaug[simi,temp_indx[0],temp_indx[1]][0])
    mccs.append(mcc_combssaug[simi,temp_indx[0],temp_indx[1]][0])
    tps.append(tp_combssaug[simi,temp_indx[0],temp_indx[1]][0])
    fps.append(fp_combssaug[simi,temp_indx[0],temp_indx[1]][0])
    bias_infs.append(biasend_inf_combssaug[simi,temp_indx[0],temp_indx[1]][0])
    biasl2s.append(biasend_l2_combssaug[simi,temp_indx[0],temp_indx[1]][0])
    tests.append(np.min(temp[temp>0]))
    ridges.append(grid_combssaug_l2[temp_indx[1]][0])
    testsamps.append(errortestendsamp_combssaug[simi,temp_indx[0],temp_indx[1]][0])

labels = ["\|B\|_0", "MCC", "TPs", "FPs", "Bias (L2)", "Test Error", "Optimal Ridge Penalty" ]
j = 0
for i in [b0s,mccs,tps,fps ,biasl2s,tests,ridges]:
    prnt_elem = i
    print(labels[j]+": "+str(np.round(np.mean(prnt_elem),2))+" "+"("+str(np.round(np.sqrt(np.var(prnt_elem)/nsim),2))+")")
    j = j+1

In [ ]:
#Print results for COMBSS augmented using \tilde{theta_t} where t is the convergent point in GD.


b0s = []
mccs = []
tps = []
fps = []
bias_infs = []
biasl2s = []
tests = []
ridges = []
testsamps = []

for simi in range(nsim):
    temp = errortestsamp_combssaug[simi,:,:]
    temp_indx = np.where(temp == np.min(temp[temp>0]))
    b0s.append(b0_combssaug[simi,temp_indx[0],temp_indx[1]][0])
    mccs.append(mcc_combssaug[simi,temp_indx[0],temp_indx[1]][0])
    tps.append(tp_combssaug[simi,temp_indx[0],temp_indx[1]][0])
    fps.append(fp_combssaug[simi,temp_indx[0],temp_indx[1]][0])
    bias_infs.append(bias_inf_combssaug[simi,temp_indx[0],temp_indx[1]][0])
    biasl2s.append(bias_l2_combssaug[simi,temp_indx[0],temp_indx[1]][0])
    tests.append(np.min(temp[temp>0]))
    ridges.append(grid_combssaug_l2[temp_indx[1]][0])
    testsamps.append(errortestsamp_combssaug[simi,temp_indx[0],temp_indx[1]][0])

labels = ["\|B\|_0", "MCC", "TPs", "FPs", "Bias (L2)", "Test Error", "Optimal Ridge Penalty" ]
j = 0
for i in [b0s,mccs,tps,fps ,biasl2s,tests,ridges]:
    prnt_elem = i
    print(labels[j]+": "+str(np.round(np.mean(prnt_elem),2))+" "+"("+str(np.round(np.sqrt(np.var(prnt_elem)/nsim),2))+")")
    j = j+1

In [ ]:
#Print results for GREEDY

mccs = []
tps = []
fps = []
bias_infs = []
biasl2s = []
tests = []
ridges = []
testsamps = []

for simi in range(nsim):
    temp = errortestsamp_greedy[simi,:,:]
    temp_indx = np.where(temp == np.min(temp[temp>0]))
    mccs.append(mcc_greedy[simi,temp_indx[0],temp_indx[1]][0])
    tps.append(tp_greedy[simi,temp_indx[0],temp_indx[1]][0])
    fps.append(fp_greedy[simi,temp_indx[0],temp_indx[1]][0])
    bias_infs.append(bias_greedy_inf[simi,temp_indx[0],temp_indx[1]][0])
    biasl2s.append(bias_greedy_l2[simi,temp_indx[0],temp_indx[1]][0])
    tests.append(np.min(temp[temp>0]))
    ridges.append(grid_greedy_l2[temp_indx[1]][0])
    testsamps.append(errortestsamp_greedy[simi,temp_indx[0],temp_indx[1]][0])
                                          
labels = ["MCC", "TPs", "FPs", "Bias (L2)", "Test Error", "Optimal Ridge Penalty"]
j = 0
for i in [mccs,tps,fps ,biasl2s,tests,ridges]:                 
    prnt_elem = i
    print(labels[j]+": "+str(np.round(np.mean(prnt_elem),2))+" "+"("+str(np.round(np.sqrt(np.var(prnt_elem)/nsim),2))+")")
    j = j+1

In [ ]:
#Print results for MIO

b0s = []
mccs = []
tps = []
fps = []
bias_infs = []
biasl2s = []
tests = []
ridges = []
testsamps = []

for simi in range(nsim):
    temp = errortestsamp_mio[simi,:,:]
    temp_indx = np.where(temp == np.min(temp[temp>0]))
    b0s.append(b0_mio[simi,temp_indx[0],temp_indx[1]][0])
    mccs.append(mcc_mio[simi,temp_indx[0],temp_indx[1]][0])
    tps.append(tp_mio[simi,temp_indx[0],temp_indx[1]][0])
    fps.append(fp_mio[simi,temp_indx[0],temp_indx[1]][0])
    bias_infs.append(bias_inf_mio[simi,temp_indx[0],temp_indx[1]][0])
    biasl2s.append(bias_l2_mio[simi,temp_indx[0],temp_indx[1]][0])
    tests.append(np.min(temp[temp>0]))
    ridges.append(grid_mio_l2[temp_indx[1]][0])
    testsamps.append(errortestsamp_mio[simi,temp_indx[0],temp_indx[1]][0])
    

labels = ["\|B\|_0", "MCC", "TPs", "FPs", "Bias (L2)", "Test Error", "Optimal Ridge Penalty" ]
j = 0
for i in [b0s,mccs,tps,fps ,biasl2s,tests,ridges]:
    prnt_elem = i
    print(labels[j]+": "+str(np.round(np.mean(prnt_elem),2))+" "+"("+str(np.round(np.sqrt(np.var(prnt_elem)/nsim),2))+")")
    j = j+1